# NTK

## NB This is a 'scratch' notebook, do not use. 

This notebook repeats the KNN benchmark reported in [Smith et al.](<https://www.biorxiv.org/content/10.1101/574723v2>)

In [2]:
from importlib import reload
import os
from pathlib import Path
import subprocess
from time import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from models.ntk import NTK, tools

import models.resampling
from models.utilities import hdf_keys

In [10]:
# DATA_PATH = Path("/data/pfizer_tx")
DATA_PATH = Path("/Users/lindsayedwards/data/pfizer_tx/tasks_all_clr")
DEFAULT_DATASET_PATH = DATA_PATH/"all_clr_train_LUAD_stage.h5"
DEFAULT_HPARAMS = dict()

In [11]:
keys = hdf_keys(DEFAULT_DATASET_PATH)
test_data = {key : pd.read_hdf(DEFAULT_DATASET_PATH, key = key) for key in keys}

In [12]:
X, y = test_data['/expression'], test_data['/labels']

In [13]:
X.shape, y.shape

((542, 57992), (542,))

In [14]:
# DEFAULT_HPARAMS['C'] = [1]
MAX_N_TOT = 10
MAX_DEPTH = 10
C_VALS = [10.0 ** i for i in range(-2, 5)]

In [15]:
alg = tools.svm

In [16]:
d_max = MAX_DEPTH

In [18]:
K = np.zeros((d_max, d_max, X.shape[0], X.shape[0]))
K.shape

(10, 10, 542, 542)

In [19]:
Ks = NTK.kernel_value_batch(X, MAX_DEPTH)

ValueError: Shape of passed values is (542, 542), indices imply (542, 57992)

In [ ]:
def kernel_value_batch(X, d_max): 
    """Returns an array K of size (d_max, d_max, N, N), 
    K[i][j] is kernel value of depth i + 1 with first j layers fixed 
    """
    K = np.zeros((d_max, d_max, X.shape[0], X.shape[0]))
    for fix_dep in range(d_max):
        S = np.matmul(X, X.T)
        H = np.zeros_like(S)
        for dep in range(d_max):
            if fix_dep <= dep:
                H += S
            K[dep][fix_dep] = H
            L = np.diag(S)
            P = np.clip(np.sqrt(np.outer(L, L)), a_min = 1e-9, a_max = None)
            Sn = np.clip(S / P, a_min = -1, a_max = 1)
            S = (Sn * (math.pi - np.arccos(Sn)) + np.sqrt(1.0 - Sn * Sn)) * P / 2.0 / math.pi
            H = H * (math.pi - np.arccos(Sn)) / 2.0 / math.pi
    return K

In [21]:
def model(params):
    return LogisticRegression(C=1, solver='lbfgs', n_jobs=-1)
nestedCV = models.resampling.NestedCV(model, hparams, 2, 2)
performance, params = nestedCV.train(test_data['/expression'], test_data['/labels'])
final_performance = np.mean(performance)
print(f"Final average model performance: {final_performance}")
results.append(final_performance)
best_params.append(params)

Outer fold 1 of 2
Inner fold 1 of 2
Fitting model with params: {'C': 0.3}
Fitting model with params: {'C': 0.6}
Inner fold 2 of 2
Fitting model with params: {'C': 0.3}
Fitting model with params: {'C': 0.6}
Best params: {'C': 0.3}, training final model
Outer fold 2 of 2
Inner fold 1 of 2
Fitting model with params: {'C': 0.3}
Fitting model with params: {'C': 0.6}
Inner fold 2 of 2
Fitting model with params: {'C': 0.3}
Fitting model with params: {'C': 0.6}
Best params: {'C': 0.3}, training final model
Total time taken: 92.77150750160217
Mean performance across 2 outer splits: 0.6174454980302057
Final average model performance: 0.6174454980302057


In [17]:
# reload(models.resampling)

<module 'models.resampling' from '/Users/lindsayedwards/Tresors/DeepLearning/tx-metalearning/models/resampling.py'>